## Load training data

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import random
from itertools import repeat
from IPython.display import display, clear_output, set_matplotlib_formats
from pandas import DataFrame

from model.pre_process import pre_process
from model.segment_normaliser import normaliser_for_seg

%load_ext autoreload
%autoreload 2
%matplotlib inline
font_size = 18
plt.rcParams.update({'font.size': font_size})
set_matplotlib_formats('pdf', quality=90)
n_hyperparam_trajs = 100
n_train_trajs = 60

plt.style.use('seaborn')
np.random.seed(1)
trajs_pickle_path = '../data/train3.pkl'
data = pd.read_pickle(trajs_pickle_path)
n_trajs = list(data.traj.unique())
random.shuffle(n_trajs)
train_traj_ids = frozenset(n_trajs[:n_train_trajs])
hyperparam_traj_ids = frozenset(n_trajs[n_train_trajs:(n_train_trajs + n_hyperparam_trajs)])
hyperparam_trajs = data[data.traj.transform(lambda j: j in hyperparam_traj_ids)]
train_trajs = data[data.traj.transform(lambda j: j in train_traj_ids)]
train_traj_ids = train_trajs.traj.unique()
train_seg_ids  = train_trajs.seg.unique()

# Segment normaliser
seg_dict = dict(tuple(data.groupby('seg')))
seg_normalisers = {
    seg_n: normaliser_for_seg(seg_dict[seg_n])
    for seg_n in seg_dict
}


def load_seg(df: DataFrame, seg_n: int, traj_n: int, frac_observed=1):
    print('loading segment', seg_n, traj_n, frac_observed)
    return pre_process(
        df[(df.traj == traj_n) & (df.seg == seg_n)], 
        stop_compress_delta, 
        seg_normalisers[seg_n],
        frac_observed
    )

def from_km(km):
    return km*1000
    
def from_km_h(km_h):
    """To m/s"""
    return km_h/3.6

stop_compress_delta  = 4 # meters
f_p_codomain = ['x', 'y']
f_v_codomain = ['dx', 'dy']
f_p_sigma_n =  .1  # meters
f_v_sigma_n = from_km_h(.00001) # m/s
g_sigma_n = .0001 # tau is deterministic
h_sigma_n = 1 # seconds
delta_xy = 4 # metres, spatial cluster width
delta_p = 4#  metres, p cluster width
delta_v = from_km_h(.5)   # metres/second, v cluster width
route_n = 3
hyp_traj_ids = hyperparam_trajs.traj.unique()
hyp_seg_ids  = hyperparam_trajs.seg.unique()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Arrival time distributions

In [40]:
from model.plotting import plot_grid, default_color
from model.pre_process import compute_time_left

h, w = 4, 3
_, axs = plot_grid(h, w)
n_segs = range(1, 12)
stds = {}
for seg_n in n_segs:
    i = seg_n - 1
    ax = axs[i // w][i % w]
    ax.set_title('Travel times of segment ' + str(seg_n), fontsize=font_size)
    ax.set_xlabel('Travel time', fontsize=font_size- 2)
    ax.set_ylabel('Frequency', fontsize=font_size - 2)
    travel_times = []
    for traj_id in train_traj_ids:
        traj = train_trajs[(train_trajs.seg == seg_n) & (train_trajs.traj == traj_id)]
        travel_time = compute_time_left(traj).time_left.iloc[0]
        travel_times.append(travel_time)
    
    ax.hist(travel_times, bins=50)
    stds[seg_n] = travel_times
    

<Figure size 1728x2304 with 12 Axes>

In [44]:
for seg_n in n_segs:
    print(np.array(stds[seg_n]).std())

40.261313109899106
13.199368671771134
14.790866175522725
12.506120723691883
11.95030218120957
12.704974966086667
21.13714476671078
12.627307269916612
25.85179985137506
24.920651855216168
20.76168345775457


## Estimate Priors

In [8]:
from model.function_model import learn_function
from model.trajectory_model import learn_model
import GPy

rbf_kernel = lambda: GPy.kern.RBF(1) 
matern_kernel = lambda: GPy.kern.Matern32(1) 
fpx_ls, fpy_ls, fvx_ls, fvy_ls = [], [], [], []
fpx_var, fpy_var, fvx_var, fvy_var = [], [], [], []

ls = np.ndarray((6, len(hyp_seg_ids), len(hyp_traj_ids)))
vars = np.ndarray((6, len(hyp_seg_ids), len(hyp_traj_ids)))
#lins = np.ndarray((2, len(hyp_seg_ids), len(hyp_traj_ids)))
for i, seg_id in enumerate(hyp_seg_ids):
    for j, traj_id in enumerate(hyp_traj_ids):
        seg = load_seg(hyperparam_trajs, seg_id, traj_id)
        normaliser = seg_normalisers[seg_id]
        scaled_f_p_sigma_n =  f_p_sigma_n / normaliser.p_scale 
        scaled_f_v_sigma_n = f_v_sigma_n / normaliser.v_scale
        scaled_delta_xy = delta_xy / normaliser.p_scale
        tau = seg.tau.values.reshape(-1, 1)
        x = seg.x.values.reshape(-1, 1)
        y = seg.y.values.reshape(-1, 1)
        dx = seg.dx.values.reshape(-1, 1)
        dy = seg.dy.values.reshape(-1, 1)
        time_left = seg.time_left.values.reshape(-1, 1)
        fpx = learn_function(tau, x, rbf_kernel(), scaled_f_p_sigma_n, n_restarts=3)
        ls[0, i, j] = fpx.model.rbf.lengthscale
        vars[0, i, j] = fpx.model.rbf.variance
        
        fpy = learn_function(tau, y, rbf_kernel(), scaled_f_p_sigma_n, n_restarts=3)
        ls[1, i, j] = fpy.model.rbf.lengthscale
        vars[1, i, j] = fpy.model.rbf.variance
        
        fvx = learn_function(tau, dx, matern_kernel(), scaled_f_v_sigma_n, n_restarts=3)
        ls[2, i, j] = fvx.model.kern.lengthscale
        vars[2, i, j] = fvx.model.kern.variance
        
        fvy = learn_function(tau, dy, matern_kernel(), scaled_f_v_sigma_n, n_restarts=3)
        ls[3, i, j] = fvy.model.kern.lengthscale
        vars[3, i, j] = fvy.model.kern.variance

        g = learn_function(np.hstack((x, y)), tau, GPy.kern.RBF(2), g_sigma_n, n_restarts=3)
        ls[4, i, j] = g.model.rbf.lengthscale
        vars[4, i, j] = g.model.rbf.variance
        #lins[0, i, j] = g.model.sum.linear.variances
        
        h = learn_function(tau, time_left, GPy.kern.RBF(1), h_sigma_n, n_restarts=3)
        ls[5, i, j] = h.model.rbf.lengthscale
        vars[5, i, j] = h.model.rbf.variance
        #lins[1, i, j] = h.model.sum.linear.variances

import pickle as pkl

with open('ls.pkl', 'wb') as handle:
    pickle.dump(ls, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('vars.pkl', 'wb') as handle:
    pickle.dump(vars, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('lins.pkl', 'wb') as handle:
    pickle.dump(lins, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

loading segment 1 22 1
Optimization restart 1/3, f = -93.5053889167191
Optimization restart 2/3, f = -93.50538890436106
Optimization restart 3/3, f = -93.50538891757935
Optimization restart 1/3, f = -105.66988795017332
Optimization restart 2/3, f = -105.66988794901337
Optimization restart 3/3, f = -105.66988795017271
Optimization restart 1/3, f = -21.23381816599333
Optimization restart 2/3, f = -21.23381816598916
Optimization restart 3/3, f = -21.233818165985017
Optimization restart 1/3, f = -103.22738577328748
Optimization restart 2/3, f = -103.22738577307558
Optimization restart 3/3, f = -103.22738577328607
Optimization restart 1/3, f = -108.85875837765674
Optimization restart 2/3, f = -108.85875837793725
Optimization restart 3/3, f = -108.85875837790206
Optimization restart 1/3, f = 56.047172118628
Optimization restart 2/3, f = 56.047172118614384
Optimization restart 3/3, f = 56.04717213880144
loading segment 1 100 1
Optimization restart 1/3, f = -99.6924816016654
Optimization resta

Optimization restart 1/3, f = -105.57936444999812
Optimization restart 2/3, f = -105.5793644491095
Optimization restart 3/3, f = -105.57936444999953
Optimization restart 1/3, f = -121.8517240297895
Optimization restart 2/3, f = -121.85172402946796
Optimization restart 3/3, f = -121.85172402978947
Optimization restart 1/3, f = -47.23194654776754
Optimization restart 2/3, f = -47.23194654778039
Optimization restart 3/3, f = -47.231946547782016
Optimization restart 1/3, f = -121.74390958870968
Optimization restart 2/3, f = -121.74390958873371
Optimization restart 3/3, f = -121.74390958862324
Optimization restart 1/3, f = -136.4103214690249
Optimization restart 2/3, f = -136.41032146899627
Optimization restart 3/3, f = -136.410321469026
Optimization restart 1/3, f = 63.79412336229542
Optimization restart 2/3, f = 63.79412336216956
Optimization restart 3/3, f = 63.79412336218978
loading segment 1 496 1
Optimization restart 1/3, f = -114.4966699663884
Optimization restart 2/3, f = -114.49666

loading segment 1 791 1
Optimization restart 1/3, f = -102.99605868294663
Optimization restart 2/3, f = -102.9960586830965
Optimization restart 3/3, f = -102.99605868310269
Optimization restart 1/3, f = -116.06528778560396
Optimization restart 2/3, f = -116.0652877856025
Optimization restart 3/3, f = -116.06528778560036
Optimization restart 1/3, f = -39.01369662676649
Optimization restart 2/3, f = -39.01369662679428
Optimization restart 3/3, f = -39.013696626774916
Optimization restart 1/3, f = -120.44317228269972
Optimization restart 2/3, f = -120.44317228316265
Optimization restart 3/3, f = -120.44317228317078
Optimization restart 1/3, f = -123.52623264786052
Optimization restart 2/3, f = -123.52623253277388
Optimization restart 3/3, f = -123.52623264789712
Optimization restart 1/3, f = 61.226619125376985
Optimization restart 2/3, f = 61.22661912357028
Optimization restart 3/3, f = 61.22661912415249
loading segment 1 866 1
Optimization restart 1/3, f = -101.7952775362659
Optimization

Optimization restart 1/3, f = -114.91670971278606
Optimization restart 2/3, f = -114.91670971281779
Optimization restart 3/3, f = -114.9167097128225
Optimization restart 1/3, f = -130.5033197553245
Optimization restart 2/3, f = -130.50331975532472
Optimization restart 3/3, f = -130.50331975491252
Optimization restart 1/3, f = -46.38114331038007
Optimization restart 2/3, f = -46.38114331035375
Optimization restart 3/3, f = -46.3811433103892
Optimization restart 1/3, f = -142.6617676678057
Optimization restart 2/3, f = -142.6617676677418
Optimization restart 3/3, f = -142.66176766780978
Optimization restart 1/3, f = -146.7610469265704
Optimization restart 2/3, f = -146.76104692664165
Optimization restart 3/3, f = -146.76104692661795
Optimization restart 1/3, f = 67.65058100393662
Optimization restart 2/3, f = 67.65058100002352
Optimization restart 3/3, f = 67.65058100251211
loading segment 1 1178 1
Optimization restart 1/3, f = -114.90279116963944
Optimization restart 2/3, f = -114.90279

Optimization restart 2/3, f = -127.04285837458907
Optimization restart 3/3, f = -127.04285837458897
Optimization restart 1/3, f = -48.58174928696417
Optimization restart 2/3, f = -48.58174928711591
Optimization restart 3/3, f = -48.5817492870993
Optimization restart 1/3, f = -166.6643620825849
Optimization restart 2/3, f = -166.66436208258412
Optimization restart 3/3, f = -166.66436208253748
Optimization restart 1/3, f = -138.5713896433025
Optimization restart 2/3, f = -138.57138964321047
Optimization restart 3/3, f = -138.57138964343613
Optimization restart 1/3, f = 72.12006147823186
Optimization restart 2/3, f = 72.120061478233
Optimization restart 3/3, f = 72.120061478227
loading segment 1 1623 1
Optimization restart 1/3, f = -111.18842988304422
Optimization restart 2/3, f = -111.1884298831225
Optimization restart 3/3, f = -111.18842988309761
Optimization restart 1/3, f = -120.28889635547748
Optimization restart 2/3, f = -120.28889635547799
Optimization restart 3/3, f = -120.2888963

...

Optimization restart 3/3, f = -167.73446429827356
Optimization restart 1/3, f = -133.80331148685323
Optimization restart 2/3, f = -133.80331148687313
Optimization restart 3/3, f = -133.80331148685102
Optimization restart 1/3, f = 97.91508504602987
Optimization restart 2/3, f = 97.91508099295483
Optimization restart 3/3, f = 97.91508099361435
loading segment 11 2601 1
Optimization restart 1/3, f = -88.24200721825787
Optimization restart 2/3, f = -88.24200721874256
Optimization restart 3/3, f = -88.24200721872941
Optimization restart 1/3, f = -97.98015145400338
Optimization restart 2/3, f = -97.98015145458639
Optimization restart 3/3, f = -97.98015145459495
Optimization restart 1/3, f = -54.17359702759693
Optimization restart 2/3, f = -54.173597029745416
Optimization restart 3/3, f = -54.17359702130107
Optimization restart 1/3, f = -184.5354098994729
Optimization restart 2/3, f = -184.5354098993577
Optimization restart 3/3, f = -184.53540989947433
Optimization restart 1/3, f = -111.82512

Optimization restart 3/3, f = -206.54907182775426
Optimization restart 1/3, f = -145.4660072593504
Optimization restart 2/3, f = -145.46600725936833
Optimization restart 3/3, f = -145.46600725820332
Optimization restart 1/3, f = 91.81449443843874
Optimization restart 2/3, f = 91.81449443841348
Optimization restart 3/3, f = 91.8144944389938
loading segment 11 2873 1
Optimization restart 1/3, f = -96.95592161924097
Optimization restart 2/3, f = -96.95592161981321
Optimization restart 3/3, f = -96.95592161968008
Optimization restart 1/3, f = -106.79990631389833
Optimization restart 2/3, f = -106.79990630583757
Optimization restart 3/3, f = -106.79990631389776
Optimization restart 1/3, f = -57.248904629164116
Optimization restart 2/3, f = -57.24890462916329
Optimization restart 3/3, f = -57.24890462868909
Optimization restart 1/3, f = -196.54947792244863
Optimization restart 2/3, f = -196.54947792290673
Optimization restart 3/3, f = -196.54947792283724
Optimization restart 1/3, f = -121.38

Optimization restart 1/3, f = -135.15353345009405
Optimization restart 2/3, f = -135.1535334518748
Optimization restart 3/3, f = -135.15353345193103
Optimization restart 1/3, f = 170.18251319899355
Optimization restart 2/3, f = 112.56179582545454
Optimization restart 3/3, f = 112.56179582545134
loading segment 11 2997 1
Optimization restart 1/3, f = -111.3311310004077
Optimization restart 2/3, f = -111.33113099913133
Optimization restart 3/3, f = -111.3311310004116
Optimization restart 1/3, f = -137.29847121755532
Optimization restart 2/3, f = -137.29847121747667
Optimization restart 3/3, f = -137.29847121755606
Optimization restart 1/3, f = -0.5140264820755256
Optimization restart 2/3, f = -0.5140264820755007
Optimization restart 3/3, f = -0.5140264820750744
Optimization restart 1/3, f = -186.50597937713053
Optimization restart 2/3, f = -186.50597937716537
Optimization restart 3/3, f = -186.50597937716648
Optimization restart 1/3, f = -144.28587735867504
Optimization restart 2/3, f = 

Optimization restart 1/3, f = -139.34070834603452
Optimization restart 2/3, f = -139.34070834761593
Optimization restart 3/3, f = -139.34070834244739
Optimization restart 1/3, f = 130.11186449795605
Optimization restart 2/3, f = 130.1118644979545
Optimization restart 3/3, f = 130.1118644979563
loading segment 11 3287 1
Optimization restart 1/3, f = -125.00799623498182
Optimization restart 2/3, f = -125.00799623497629
Optimization restart 3/3, f = -127.25106670899969
Optimization restart 1/3, f = -155.24290108770427
Optimization restart 2/3, f = -155.24290108770558
Optimization restart 3/3, f = -155.24290108766382
Optimization restart 1/3, f = -28.839176777346072
Optimization restart 2/3, f = -28.839176777345983
Optimization restart 3/3, f = -28.839176777345948
Optimization restart 1/3, f = -238.3995922724044
Optimization restart 2/3, f = -238.3995922722514
Optimization restart 3/3, f = -238.39959227242343
Optimization restart 1/3, f = -158.94060253961598
Optimization restart 2/3, f = -

Optimization restart 3/3, f = -214.5213150281402
Optimization restart 1/3, f = -154.9668249617077
Optimization restart 2/3, f = -154.96682496190806
Optimization restart 3/3, f = -154.9668249619193
Optimization restart 1/3, f = 118.63873811789414
Optimization restart 2/3, f = 118.63873811835677
Optimization restart 3/3, f = 118.63873811802338
loading segment 11 3598 1
Optimization restart 1/3, f = -109.66931753542137
Optimization restart 2/3, f = -109.66931753567698
Optimization restart 3/3, f = -109.66931753524777
Optimization restart 1/3, f = -126.62010962815265
Optimization restart 2/3, f = -126.62010962813892
Optimization restart 3/3, f = -126.62010962815259
Optimization restart 1/3, f = -9.245067816714268
Optimization restart 2/3, f = -9.245067816714052
Optimization restart 3/3, f = -9.245067816584989
Optimization restart 1/3, f = -180.71860764944552
Optimization restart 2/3, f = -180.71860765070056
Optimization restart 3/3, f = -180.71860765075573
Optimization restart 1/3, f = -13

In [9]:
plt.hist(ls[1][0])

(array([ 7., 20., 31., 21.,  5.,  6.,  1.,  3.,  3.,  3.]),
 array([0.17941661, 0.2003696 , 0.22132259, 0.24227558, 0.26322857,
        0.28418156, 0.30513455, 0.32608754, 0.34704053, 0.36799352,
        0.38894651]),
 <a list of 10 Patch objects>)

<Figure size 576x396 with 1 Axes>

In [ ]:
fpx_ls, fpy_ls, fvx_ls, fvy_ls = [], [], [], []
fpx_var, fpy_var, fvx_var, fvy_var = [], [], [], []
for seg_models in models:
    

fpx_ls.append(float(f_p_x.model.rbf.variance))
fpx_var.append(float(f_p_x.model.rbf.lengthscale))

fpy_ls.append(float(f_p_y.model.rbf.variance))
fpy_var.append(float(f_p_y.model.rbf.variance))

fvx_ls.append(float(f_v_x.model.kern.variance))
fvx_var.append(float(f_p_y.model.kern.variance))

fvy_ls.append(float(f_v_y.model.kern.variance))
fvy_var.append(float(f_v_y.model.kern.variance))

In [10]:
print(len(hyp_traj_ids))
print(len(hyp_seg_ids))
from model.plotting import plot_grid, default_color
from scipy.stats import gamma

plt.style.use('seaborn')
plt.rcParams.update({'font.size': 18})

def plot_estimated_prior(ax, plot_range, hist_data, quantity, fit_color, f, unit):
    ax.hist(hist_data, range=plot_range, bins=50, density=True, color=hist_color)
    fit_alpha, fit_loc, fit_beta = gamma.fit(hist_data, floc=0)
    xx = np.linspace(plot_range[0], plot_range[1], 200)
    yy = gamma.pdf(xx, fit_alpha, fit_loc, fit_beta)
    ax.plot(xx, yy, label='$p($' + quantity +'$)$', color=fit_color)
    ax.set_xlabel(quantity + ' (' + unit +')')
    ax.set_xlim(plot_range)
    ax.set_title(r'Histogram of ' + quantity + ' for $' + f +'$')
    ax.set_ylabel(r'Density')
    ax.legend()
    print('${}$ {} parameters: $\\alpha$={} $\\beta$={}'.format(f, quantity,fit_alpha, fit_beta))

    
_, axs = plot_grid(2, 4 )
fpx_ell_ax = axs[0, 0]
fpx_var_ax = axs[1, 0]
fpy_ell_ax = axs[0, 1]
fpy_var_ax = axs[1, 1]
fvx_ell_ax = axs[0, 2]
fvx_var_ax = axs[1, 2]
fvy_ell_ax = axs[0, 3]
fvy_var_ax = axs[1, 3]

resolution = 300
hist_color = default_color(1)
ell_color = default_color(2)
var_color = default_color(4)
ell_hist_label = r'$\ell$'
ell_fit_label = r'Estiamted $p(\ell)$'

seg_n = 0
fpx_range = (0, 1)
plot_estimated_prior(fpx_ell_ax, fpx_range, ls[0, seg_n], ell_hist_label, ell_color, 'f_{p_x}', 'metres')
fpy_range = (0, 1)
plot_estimated_prior(fpy_ell_ax, fpy_range, ls[1, seg_n], ell_hist_label, ell_color, 'f_{p_y}', 'metres')
fvx_range = (0, 1)
fvx_ls_kmh = np.array(fvx_ls)*3.6
plot_estimated_prior(fvx_ell_ax, fvx_range, ls[2, seg_n], ell_hist_label, ell_color, 'f_{v_x}', 'km/h')
fvy_range = (0, .5)
fvy_ls_kmh = np.array(fvy_ls)*3.6
plot_estimated_prior(fvy_ell_ax, fvy_range, ls[3, seg_n], ell_hist_label, ell_color, 'f_{v_y}', 'km/h')

var_hist_label = r'$\sigma^2$'

fpx_var_range = (0, 12)
plot_estimated_prior(fpx_var_ax, fpx_var_range, vars[0, seg_n], var_hist_label, var_color, 'f_{p_x}', 'metres')
fpy_var_range = (0, .1)
plot_estimated_prior(fpy_var_ax, fpy_var_range, vars[1, seg_n], var_hist_label, var_color, 'f_{p_y}', 'metres')
fvx_var_range = (0, 7)
fvx_var_kmh = np.array(fvx_var)*3.6
plot_estimated_prior(fvx_var_ax, fvx_var_range, vars[2, seg_n], var_hist_label, var_color, 'f_{v_x}', 'metres')

fvy_var_range = (0, 0.05)
fvy_var_kmh = np.array(fvy_var)*3.6
plot_estimated_prior(fvy_var_ax, fvy_var_range, vars[3, seg_n], var_hist_label, var_color, 'f_{v_y}', 'metres')

100
11
$f_{p_x}$ $\ell$ parameters: $\alpha$=16.09090289562399 $\beta$=0.02097682347161703
$f_{p_y}$ $\ell$ parameters: $\alpha$=35.83911007067104 $\beta$=0.006914971605915522
$f_{v_x}$ $\ell$ parameters: $\alpha$=6.590096047791525 $\beta$=0.043010920742238314
$f_{v_y}$ $\ell$ parameters: $\alpha$=11.717980293137398 $\beta$=0.013799158395195767
$f_{p_x}$ $\sigma^2$ parameters: $\alpha$=10.23704313364959 $\beta$=0.36124060499263133
$f_{p_y}$ $\sigma^2$ parameters: $\alpha$=111.64940249020235 $\beta$=0.0003144241361540167
$f_{v_x}$ $\sigma^2$ parameters: $\alpha$=4.152322446572946 $\beta$=0.5633174341301
$f_{v_y}$ $\sigma^2$ parameters: $\alpha$=13.466672751274345 $\beta$=0.0008956581973416192


<Figure size 2304x1152 with 8 Axes>

$f_{p_x}$ $\ell$ parameters: $\alpha$=0.4535984866180475 $\beta$=5.95689466158176

$f_{p_y}$ $\ell$ parameters: $\alpha$=0.1497037731343836 $\beta$=3.95908043440209

$f_{v_x}$ $\ell$ parameters: $\alpha$=2.1375357609511085 $\beta$=1.3298621485622508

$f_{v_y}$ $\ell$ parameters: $\alpha$=0.7728656323799816 $\beta$=0.013162814492013066

$f_{p_x}$ $\sigma^2$ parameters: $\alpha$=2.0166671257004736 $\beta$=0.43416059060187423

$f_{p_y}$ $\sigma^2$ parameters: $\alpha$=0.1497037731343836 $\beta$=3.95908043440209

$f_{v_x}$ $\sigma^2$ parameters: $\alpha$=0.14970377313438363 $\beta$=14.25268956384752

$f_{v_y}$ $\sigma^2$ parameters: $\alpha$=0.7728656323799816 $\beta$=0.013162814492013066

In [11]:
from scipy.stats import gamma
import pickle

#alpha_l, beta_l, alpha_var, beta_var, alpha_lin, beta_var
#ls = np.ndarray((6, len(hyp_seg_ids), len(hyp_traj_ids)))
#vars = np.ndarray((6, len(hyp_seg_ids), len(hyp_traj_ids)))
#lins = np.ndarray((2, len(hyp_seg_ids), len(hyp_traj_ids)))
def fit_gammas(data):
    n_segs, n_models, n_params = len(hyp_seg_ids), 6, 2
    params = np.ndarray((n_segs, n_models, n_params))
    for i, seg_id in enumerate(hyp_seg_ids):
        fpx_l_alpha, _, fpx_l_beta = gamma.fit(data[0, i], floc=0)
        fpy_l_alpha, _, fpy_l_beta = gamma.fit(data[1, i], floc=0)
        fvx_l_alpha, _, fvx_l_beta = gamma.fit(data[2, i], floc=0)
        fvy_l_alpha, _, fvy_l_beta = gamma.fit(data[3, i], floc=0)
        g_l_alpha, _, g_l_beta = gamma.fit(data[4, i], floc=0)
        h_l_alpha, _, h_l_beta = gamma.fit(data[5, i], floc=0)
        params[i, 0, 0] = fpx_l_alpha
        params[i, 0, 1] = fpx_l_beta
        params[i, 1, 0] = fpy_l_alpha
        params[i, 1, 1] = fpy_l_beta
        params[i, 2, 0] = fvx_l_alpha
        params[i, 2, 1] = fvx_l_beta
        params[i, 3, 0] = fvy_l_alpha
        params[i, 3, 1] = fvy_l_beta
        params[i, 4, 0] = g_l_alpha
        params[i, 4, 1] = g_l_beta
        params[i, 5, 0] = h_l_alpha
        params[i, 5, 1] = h_l_beta
    return params

with open('ls.pkl', 'rb') as handle:
    ls = pickle.load(handle)

with open('vars.pkl', 'rb') as handle:
    vars = pickle.load(handle)

ell_hyperparams = fit_gammas(ls)
var_hyperparams = fit_gammas(vars)

with open('ell-hyperparams.pkl', 'wb') as handle:
    pickle.dump(ell_hyperparams, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('var-hyperparams.pkl', 'wb') as handle:
    pickle.dump(var_hyperparams, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [12]:

with open('lin-hyperparams.pkl', 'rb') as handle:
    lin_hyperparams = pickle.load(handle)
    
    
n_segs, n_models, n_params = len(hyp_seg_ids), 2, 2
lin_params = np.ndarray((n_segs, n_models, n_params))
for i, seg_id in enumerate(hyp_seg_ids):
        g_l_alpha, _, g_l_beta = gamma.fit(lins[0, i], floc=0)
        h_l_alpha, _, h_l_beta = gamma.fit(lins[1, i], floc=0)
        lin_params[i, 0, 0] = g_l_alpha
        lin_params[i, 0, 1] = g_l_beta
        lin_params[i, 1, 0] = h_l_alpha
        lin_params[i, 1, 1] = h_l_beta

with open('lin-hyperparams.pkl', 'wb') as handle:
    pickle.dump(lin_params, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

 C:\Users\sebas\Anaconda3\envs\gpflow\lib\site-packages\scipy\stats\_continuous_distns.py:2486: RuntimeWarning:invalid value encountered in less_equal


FitDataError: Invalid values in `data`.  Maximum likelihood estimation with 'gamma' requires that 0 < x < inf for each x in `data`.

In [13]:
#lin_params[0]
plt.hist(vars[0][1], bins=50)

(array([68., 18.,  6.,  1.,  2.,  0.,  1.,  0.,  1.,  2.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 1.17343087,  2.85833414,  4.54323741,  6.22814068,  7.91304395,
         9.59794722, 11.2828505 , 12.96775377, 14.65265704, 16.33756031,
        18.02246358, 19.70736685, 21.39227013, 23.0771734 , 24.76207667,
        26.44697994, 28.13188321, 29.81678648, 31.50168976, 33.18659303,
        34.8714963 , 36.55639957, 38.24130284, 39.92620611, 41.61110939,
        43.29601266, 44.98091593, 46.6658192 , 48.35072247, 50.03562574,
        51.72052902, 53.40543229, 55.09033556, 56.77523883, 58.4601421 ,
        60.14504538, 61.82994865, 63.51485192, 65.19975519, 66.88465846,
        68.56956173, 70.25446501, 71.93936828, 73.62427155, 75.30917482,
        76.99407809, 78.67898136, 80.36388464, 82.04878791,

<Figure size 576x396 with 1 Axes>